In [6]:
import os
import numpy as np
import trimesh

from scipy.spatial import cKDTree
from collections import Counter
from pyproj import CRS, Transformer
from scipy.spatial.distance import cosine

In [7]:
mesh_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/Buildings3D_2023January3/OID_1024/esriGeometryMultiPatch.obj'
mesh = trimesh.load_mesh(mesh_file)

In [8]:
# mesh.merge_vertices()
vertices = mesh.vertices
# print(mesh.edges)
# print(vertices)
print('-'*30 + ' Abstract ' + '-'*30)
print('The number of edges: ', len(mesh.edges_face))
print('The number of vertices: ', len(mesh.vertices))
print('The number of polygons: ', len(mesh.facets))
print('The number of faces: ', len(mesh.faces))

print()
print('-'*30 + ' Details ' + '-'*30)
print('Faces: ', mesh.faces[0:3])
print()
print('Vertices: ', mesh.vertices[0:3])
print()
print('Polygons: ', mesh.facets[0:3]) # consisting of face indexes
print()
print('Edges:', mesh.edges[0:3]) # consisting of two vertice indexs

------------------------------ Abstract ------------------------------
The number of edges:  501
The number of vertices:  115
The number of polygons:  54
The number of faces:  167

------------------------------ Details ------------------------------
Faces:  [[3 2 1]
 [1 0 3]
 [7 6 5]]

Vertices:  [[ 4.17  9.65  5.73]
 [ 7.62 12.55  5.26]
 [ 7.62  0.    5.26]]

Polygons:  [array([0, 1]), array([2, 3]), array([4, 5])]

Edges: [[3 2]
 [2 1]
 [1 3]]


In [9]:
# angle_threshold = np.radians(180)
# print(angle_threshold)
facets = trimesh.graph.facets(mesh, facet_threshold=3)

print(f"共识别出 {len(facets)} 个 facet(共面区域)")

print(len(mesh.facets))

flat_list = sorted([int(item) for sublist in mesh.facets for item in sublist])
flat_list = set(flat_list)
print(len(flat_list))


flat_list = sorted([int(item) for sublist in facets for item in sublist])
flat_list = set(flat_list)
print(len(flat_list))

print('faces: ', len(mesh.faces))

共识别出 44 个 facet(共面区域)
54
136
150
faces:  167


In [ ]:
# --------------------------- Merge Vertices ---------------------------
vertices = mesh.vertices
faces = mesh.faces

# building tree
tree = cKDTree(vertices)
groups = tree.query_ball_tree(tree, 10e-2)
print(groups)

# building new a mesh model
vertex_map = np.full(len(vertices), -1, dtype=int)
print(vertex_map)
new_vertices = []
for i, group in enumerate(groups):
    if vertex_map[i] == -1:
        new_index = len(new_vertices)
        new_vertices.append(np.mean(vertices[group], axis=0))
        for idx in group:
            vertex_map[idx] = new_index

new_faces = np.array([[vertex_map[idx] for idx in face] for face in faces])

mesh = trimesh.Trimesh(vertices=np.array(new_vertices), faces=new_faces, process=False)
# new_mesh.export("/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/wireframe/merged_model.obj")

[[0, 13], [1, 15, 17, 18, 23, 56], [2, 24], [3], [4, 9], [5], [6], [7, 10], [8], [4, 9], [7, 10], [11], [12], [0, 13], [14], [1, 15, 17, 18, 25, 57], [16, 17, 26], [1, 15, 16, 17, 18, 25, 57], [1, 15, 17, 18, 23, 56], [19], [20], [21], [22, 37], [1, 18, 23, 25, 56, 57], [2, 24], [15, 17, 23, 25, 26, 56, 57], [16, 25, 26], [27], [28], [29], [30], [31], [32], [33], [34], [35, 36], [35, 36], [22, 37], [38], [39], [40], [41, 51], [42, 50], [43], [44, 52], [45], [46], [47], [48, 49], [48, 49], [42, 50], [41, 51], [44, 52], [53], [54, 58], [55], [1, 18, 23, 25, 56, 57], [15, 17, 23, 25, 56, 57], [54, 58], [59]]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [25]:
# --------------------- Coordinate Transformation -------------------
vertices = mesh.vertices

def parse_wld3(wld3_path):
    with open(wld3_path, 'r') as f:
        lines = f.readlines()

    def parse_line(line):
        parts = line.strip().split()
        from_pt = np.array([float(x) for x in parts[0].split(',')])
        to_pt = np.array([float(x) for x in parts[1].split(',')])
        return from_pt, to_pt

    p1, g1 = parse_line(lines[0])
    p2, g2 = parse_line(lines[1])
    return p1, g1, p2, g2

def compute_affine_2d(p1, g1, p2, g2):
    v_model = p2[:2] - p1[:2]
    v_geo = g2[:2] - g1[:2]

    scale = np.linalg.norm(v_geo) / np.linalg.norm(v_model)
    angle = np.arctan2(v_geo[1], v_geo[0]) - np.arctan2(v_model[1], v_model[0])

    cos_r = np.cos(angle)
    sin_r = np.sin(angle)
    transform_matrix = scale * np.array([[cos_r, -sin_r],
                                         [sin_r,  cos_r]])
    offset = g1[:2] - transform_matrix @ p1[:2]
    z_offset = g1[2] - p1[2]

    return transform_matrix, offset, z_offset

def transform_model_points_3d(points, transform_matrix, offset, z_offset):
    points = np.asarray(points)
    xy_local = points[:, :2]  # Nx2
    z_local = points[:, 2]    # N

    xy_transformed = (transform_matrix @ xy_local.T).T + offset  # Nx2
    z_transformed = z_local + z_offset

    return np.hstack([xy_transformed, z_transformed[:, None]])


wld3_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/Buildings3D_2023January3/OID_1024/esriGeometryMultiPatch.wld3'
prj_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/Buildings3D_2023January3/OID_1024/esriGeometryMultiPatch.prj'

p1, g1, p2, g2 = parse_wld3(wld3_file)
transform_matrix, offset, z_offset = compute_affine_2d(p1, g1, p2, g2)
vertices = transform_model_points_3d(vertices, transform_matrix, offset, z_offset)
print('vertices: ', vertices.shape)
print('vertices: ', (mesh.vertices).shape)


vertices:  (115, 3)
vertices:  (115, 3)


In [200]:
print(mesh.facets)
flat_list = sorted([int(item) for sublist in mesh.facets for item in sublist])
flat_set = set(flat_list)

# print(len(mesh.faces))
# print(len(flat_set))

all_faces = set(range(len(mesh.faces)))
missing_faces = sorted(list(all_faces - flat_set))

# 把每个缺失的面作为一个单独 facet
missing_facets = [[i] for i in missing_faces]

# 合并原始 facet 和新生成的
complete_facets = list(mesh.facets) + missing_facets

[array([0, 1]), array([2, 3]), array([4, 5]), array([6, 7]), array([8, 9]), array([10, 11]), array([12, 13]), array([14, 15]), array([22, 23]), array([24, 25])]


In [12]:
# ------------------- Convert Mesh to Wireframe ------------------- 
# mesh.facets(): merge the triangular faces into polygons
#                return the polygons as a list of face indexes
# --------------------------- End ---------------------------------
vertices = mesh.vertices
faces = mesh.faces

wireframe_edges = np.empty((0, 2), dtype=np.int32)
for facet in mesh.facets:
    edges = np.array([faces[f_index] for f_index in facet])
    edges = np.vstack([
        edges[:, [0 ,1]],
        edges[:, [1, 2]],
        edges[:, [2, 0]]])
    edges = np.sort(edges, axis=1)
    edge_counts = Counter(map(tuple, edges))
    edges = np.array([edge for edge, count in edge_counts.items() if count == 1])
    if edges.ndim != 1:
        wireframe_edges = np.concatenate([wireframe_edges, edges], axis=0)

# Remove duplicate edges
wireframe_edges = np.sort(wireframe_edges, axis=1)
wireframe_edges = np.unique(wireframe_edges, axis=0)
print('Wireframe edges: ', len(wireframe_edges))

# Remove duplicate vertices
wireframe_vertices_index = wireframe_edges.flatten()
wireframe_vertices_index = np.unique(wireframe_vertices_index)
wireframe_vertices_index = np.sort(wireframe_vertices_index)
print('Wireframe vertices: ', len(wireframe_vertices_index))

Wireframe edges:  191
Wireframe vertices:  110


In [18]:
# merge multiple segments in one line 
merged_edges = []
edges_list = wireframe_edges.tolist()
vertices_list = vertices.tolist()
# print('edges_list: ', edges_list)
# print(type(vertices))

while edges_list:
    edge = edges_list.pop()
    v1_idx, v2_idx = edge
    v1 = vertices_list[v1_idx]
    v2 = vertices_list[v2_idx]
    
    # check if the edge is connected to any other edges
    merged = False
    temp_merged_edges = merged_edges.copy()
    # print('temp_merged_edges: ', temp_merged_edges)
    for other_edge in temp_merged_edges:
        o1_idx, o2_idx = other_edge
        if (v1_idx not in [o1_idx, o2_idx]) and (v2_idx not in [o1_idx, o2_idx]):
            continue
        o1 = vertices_list[o1_idx]
        o2 = vertices_list[o2_idx]
    
        d1 = np.array(v2) - np.array(v1)
        d2 = np.array(o2) - np.array(o1)
        value_1 = 1 - abs(1 - cosine(d1, d2))

        vs, count = np.unique(np.array([v1, v2, o1, o2]), axis=0, return_counts=True)
        vs = vs[count == 1]
        try:
            d3 = vs[0] - vs[1]
        except:
            merged = True
        value_2 = 1 - abs(1 - cosine(d3, d2))

        # if the angle between the two edges is small enough, merge them
        if (value_1 < 0.001) and (value_2 < 0.01):
            merged_edges.remove(other_edge)
            if v1_idx in [o1_idx, o2_idx]:
                if v1_idx == o1_idx:
                    merged_edges.append((v2_idx, o2_idx))
                    v1_idx = o2_idx
                elif v1_idx == o2_idx:
                    merged_edges.append((v2_idx, o1_idx))
                    v1_idx = o1_idx
                v1 = vertices[v1_idx]
            elif v2_idx in [o1_idx, o2_idx]:
                if v2_idx == o1_idx:
                    merged_edges.append((v1_idx, o2_idx))
                    v2_idx = o2_idx
                elif v2_idx == o2_idx:
                    merged_edges.append((v1_idx, o1_idx))
                    v2_idx = o1_idx
                v2 = vertices[v2_idx]
            merged = True
        
    if not merged:
        merged_edges.append(edge)

edges_list = merged_edges

# Remove duplicate edges
wireframe_edges = np.sort(edges_list, axis=1)
wireframe_edges = np.unique(wireframe_edges, axis=0)
print('Wireframe edges: ', len(wireframe_edges))

# Remove duplicate vertices
wireframe_vertices_index = wireframe_edges.flatten()
wireframe_vertices_index = np.unique(wireframe_vertices_index)
wireframe_vertices_index = np.sort(wireframe_vertices_index)
print('Wireframe vertices: ', len(wireframe_vertices_index))

Wireframe edges:  123
Wireframe vertices:  93


/home/shangfeng/anaconda3/envs/BuildingWorld/lib/python3.9/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


In [13]:
# ------------------- Save Wireframe  -------------------
wireframe_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/wireframe/1024.obj'
with open(wireframe_file, 'w') as f:
    f.write('# OBJ file\n')
    f.write('# Vertices: '+ str(len(wireframe_vertices_index)) +'\n')
    f.write('# Edges: '+ str(len(wireframe_edges)) +'\n')
    
    for vertex_index in wireframe_vertices_index:
        f.write('v ')
        f.write(' '.join(map(str, vertices[int(vertex_index)])) + '\n')
        
    for edge in wireframe_edges:
        f.write('l ')
        f.write(str(np.where(wireframe_vertices_index == edge[0])[0][0]+1) + ' ')
        f.write(str(np.where(wireframe_vertices_index == edge[1])[0][0]+1) + '\n')

In [ ]:
# ------------------- Coordinate projection -------------------

import numpy as np
from pyproj import CRS, Transformer

def parse_wld3(wld3_path):
    with open(wld3_path, 'r') as f:
        lines = f.readlines()

    def parse_line(line):
        parts = line.strip().split()
        from_pt = np.array([float(x) for x in parts[0].split(',')])
        to_pt = np.array([float(x) for x in parts[1].split(',')])
        return from_pt, to_pt

    p1, g1 = parse_line(lines[0])
    p2, g2 = parse_line(lines[1])
    return p1, g1, p2, g2

def compute_affine_2d(p1, g1, p2, g2):
    v_model = p2[:2] - p1[:2]
    v_geo = g2[:2] - g1[:2]

    scale = np.linalg.norm(v_geo) / np.linalg.norm(v_model)
    angle = np.arctan2(v_geo[1], v_geo[0]) - np.arctan2(v_model[1], v_model[0])

    cos_r = np.cos(angle)
    sin_r = np.sin(angle)
    transform_matrix = scale * np.array([[cos_r, -sin_r],
                                         [sin_r,  cos_r]])
    offset = g1[:2] - transform_matrix @ p1[:2]
    z_offset = g1[2] - p1[2]

    return transform_matrix, offset, z_offset

def transform_model_point_3d(p, transform_matrix, offset, z_offset):
    xy_geo = transform_matrix @ p[:2] + offset
    z_geo = p[2] + z_offset
    return np.array([xy_geo[0], xy_geo[1], z_geo])

def read_crs_from_prj(prj_path):
    with open(prj_path, 'r') as f:
        wkt = f.read()
    return CRS.from_wkt(wkt)

if __name__ == '__main__':
    # 替换为你的文件路径
    wld3_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/Buildings3D_2023January3/OID_1/esriGeometryMultiPatch.wld3'
    prj_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/Buildings3D_2023January3/OID_1/esriGeometryMultiPatch.prj'

    # 解析文件和建立转换矩阵
    p1, g1, p2, g2 = parse_wld3(wld3_file)
    transform_matrix, offset, z_offset = compute_affine_2d(p1, g1, p2, g2)
    crs_proj = read_crs_from_prj(prj_file)
    crs_geo = CRS.from_epsg(4326)  # WGS84 经纬度

    transformer = Transformer.from_crs(crs_proj, crs_geo, always_xy=True)

    # 测试：模型空间中的任意点
    model_point = np.array([20, 1000, 0])

    # 变换到地图坐标系
    projected_point = transform_model_point_3d(model_point, transform_matrix, offset, z_offset)
    print("地图投影坐标 (X, Y, Z):", projected_point)

    # 转换为经纬度
    lon, lat = transformer.transform(projected_point[0], projected_point[1])
    print(f"经纬度坐标: Lat: {lat:.6f}, Lon: {lon:.6f}, Elevation: {projected_point[2]:.2f} m")
